In [ ]:
#   This program details the approach to solving for the optimal reimbursements (among other things) for the insurers in the simultaneous bargaining case with linear demand and no recapture. The way things are defined here allows for both active or passive beliefs. This is a test document to work through the approach associated with taking this simulation approach, which uses covariance_beliefs.ipynb as a reference point. 

In [ ]:
import numpy as np
from scipy.optimize import minimize,fsolve
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#   Computing the price $p_i$ offered by each insurer $i$. Since this is the linear demand case, we already have these functional forms, but in the logit case, this likely needs additional details. As calculated in the paper in closed form, these assume interior solutions.
#   In contrast to covariance_beliefs.ipynb, I use "lam" instead of "cost" to convey lambda (without the lambda command arising) and "v" instead of "wtp" to convey value v for the consumers. 
def calc_price(phi1, phi2, lam, v):
    p1 = (2 * phi1 + phi2) / 3. + lam
    p2 = (2 * phi2 + phi1) / 3. + lam
    return p1, p2
#   Computing the market share $s_i$ captured by each insurer $i$. We begin by assuming exogenous recapture, which we no longer think is the case. (Namely, this portion implements Equation (9) from the current draft, which assumes the Hotelling solution for market share.) I'll pursue endogenous recapture after successfully replicating this code. 
def calc_s(phi1, phi2, lam, v):
    p1, p2 = calc_price(phi1, phi2, lam, v)
    t_hat = .5 + (p2 - p1) / (2. * lam)
    return t_hat
#   Computing the profits for each insurer. Since we are looking at the simultaneous case, insurers are symmetric, which simplifies items. "mc" corresponds to the cost associated with the hospital of processing claims associated with each insurer. In this case, mc is assumed to be the same for both insurers. 
def calc_profits(phi1, phi2, lam, v, mc):
    #   This is the market share detailed in the previous line. Market share is calculated for insurer 1, with insurer 2 assumed to capture the remainder in this case, where all consumers buy insurance. 
    s = calc_s(phi1, phi2, lam, v)
    #   These are the prices detailed previously. 
    p1, p2 = calc_price(phi1, phi2, lam, v)
    #   These are the profits for each insurer. The comma allows us to define both in-line.
    profits1, profits2 = s * (p1 - phi1), (1-s) * (p2 - phi2)
    #   This is the hospital profit. By assumption, the market is fully captured by the two insurers.
    hosp_profit = s * (phi1 - mc1) + (1-s) * (phi2 - mc2)
    return hosp_profit, profits1, profits2
#   Computing the disagreement payoff $\pi_h(\mathcal{G}\setminus i)$. Note that we have assumed that $v$ is sufficiently high, which we may need to relax later. 
def disagreement_simult(phi1, phi2, lam, v, mc, active = False):
    #   Cost of each insurer to the hospital is the same. 
    mc1, mc2 = mc
    #   Without recapture:
    s_hat = calc_s(phi1, phi2, lam, v)
    p1, p2 = calc_price(phi1, phi2, lam, v)
    #   With recapture, we have the following. "np.clip" limits the values in an array, namely bounding the share between 0 and 1. I'm not entirely sure how this captures the idea with recapture, though. 
    s_hat = np.clip((lam + p2 - v) / lam, 0, 1)
    if active